In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [3]:
import matplotlib
import signale
import numpy as np
import matplotlib.pyplot as pl
import spectrum as sp
import pickle as pkl
import scipy.signal as scsig
import pywt
import colormaps as mycmps
import signale.tools as tools
import os
import itertools
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from sklearn.cluster import KMeans
from matplotlib.mlab import PCA as mlabPCA
import mtspec

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
def MA(array,ord=2):
    ma = np.array([array[ii-ord/2:ii+ord/2].mean() for ii in range(array.size) if ii >= ord ],dtype=float)
    ma = np.insert(ma,0,array[0:ord/2])
    ma = np.append(ma,array[-(ord/2)-1:-1])
    return ma
def zScore(array):
    return(array - np.average(array))/ np.std(array)
def mySpecgram(sig,minFreq = None,maxFreq = None,overlap=None,
               windowSize= None,NFFT = None,Fs = None,method=None):
    if not overlap:
        overlap = 48
    kIdx = 0
    timeSlices = []
    timeCenters = []
    while kIdx < sig.size:
        #timeEdges = np.arange(0,Nbins*windowSize,windowSize)
        timeSlices.append([kIdx,kIdx+windowSize])
        kIdx += windowSize - overlap
    Pxx = []
    p = sp.Periodogram(sig[timeSlices[0][0]:timeSlices[0][1]],NFFT=NFFT,sampling=Fs)
    freqz = np.array(p.frequencies())
    for item in timeSlices: 
        timeCenters.append((item[0]+np.diff(item)/2)[0])
        data = sig[item[0]:item[1]]
        if data.size < windowSize:
            ddd = windowSize - data.size
            if ddd%2 ==0:
                data = np.lib.pad(data, (ddd/2,ddd/2), 'constant', constant_values=(0,0))
            else:
                data = np.lib.pad(data, (ddd/2,ddd/2+1), 'constant', constant_values=(0,0))
        if method == 'yule':
            p = sp.pyule(data, 550, norm='biased', NFFT=NFFT,sampling=Fs)
        elif method =='fft':
            p = sp.Periodogram(data,NFFT=NFFT,sampling=Fs)
        elif method =='mpm':
            [tapers, eigen] = sp.dpss(windowSize, 3, 6)
            res = sp.pmtm(data, e=tapers, v=eigen, show=False)
            res = res.flatten()
        p();
        if method=='mpm':
            Pxx.append(res[:freqz.size])
        else:
            Pxx.append(p.psd)
    Pxx = np.array(Pxx).T
    if not minFreq:
        minFreq = freqz.min()
    if not maxFreq:
        maxFreq = freqz.max()
    indexstart = np.where(freqz >= minFreq)[0][0]
    indexend   = np.where(freqz <= maxFreq)[0][-1]
    freqz = freqz[indexstart:indexend]
    Pxx = Pxx[indexstart:indexend]
    timeCenters = np.array(timeCenters)
    return timeCenters,freqz,Pxx


In [5]:
animalPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/'
ddayx = animalPath.split('/')[-2].split('_')[-1]
print ddayx[-1]
dataFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/trajectory/'

1


In [6]:
lfpPaths = []
for lfp in tools.locate('*CSC*lfp',animalPath):#animalPath):
     if lfp[0].find('02')>0:
            lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
ImmoDBpaths = []
for db in tools.locate('lin*.db',dataFolder):
    if db[1].find(ddayx[-1])>0:
        ImmoDBpaths.append(os.path.join(db[0],db[1]))
lfpPaths = sorted(lfpPaths)
ImmoDBpaths = sorted(ImmoDBpaths)
traj = pkl.load(open(animalPath + 'parrentData/linearTrack_Trajectory.traj','rb'))
lfpPaths,ImmoDBpaths


(['/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC01.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC02.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC03.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC04.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC05.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC06.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC07.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC08.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC09.lfp',
 

In [7]:
SleepEpochsDB = pd.DataFrame([],columns=['t0','t1','epoch','session','date','animal'])
for jtem in lfpPaths[1:2]:
    print 'LOADING %s' %jtem
    ##Loading...
    expDate = animalPath.split('/')[-2].split('_')[-1]
    session = jtem.split('/')[-2].split('-')[-1]
    session = session[:-1]+'0'+session[-1]
    animal = [prt for prt in jtem.split('/') if not prt.find('Rat')][0]
    #SleepTimeIndex = pd.read_pickle(item)
    lfp = pkl.load(open(jtem,'rb'))
    t,frq,Pxx =  mySpecgram(lfp.signal_white[:],minFreq=0.1,maxFreq=20,windowSize=1028,method='fft',Fs=2000,overlap=0,NFFT=4096)
    Pxx = np.clip(Pxx,0,Pxx.mean()+3*Pxx.std())
    pxxFlat = Pxx# np.array([Pxx[:,ii]*frq for ii in range(Pxx.shape[1])]).T
    for jjj in range(pxxFlat.shape[0]):
        pxxFlat[jjj,:] = MA(pxxFlat[jjj,:],2)
    
    ##Grouping into different freqency bands, delta,theta,gamma
    Dmean = frq[frq<3.5].mean()
    Tmean = frq[np.logical_and(frq>4,frq<8)].mean()
    Gmean = frq[frq>10].mean()
    Dbounds = np.where(frq<3.5)[0][[0,-1]]
    Tbounds = np.where(np.logical_and(frq>4,frq<8))[0][[0,-1]]
    Gbounds = np.where(frq>10)[0][[0,-1]]
    DavgPwr = Pxx[Dbounds[0]:Dbounds[-1]+1,:].mean(axis=0)
    TavgPwr = Pxx[Tbounds[0]:Tbounds[-1]+1,:].mean(axis=0)
    GavgPwr = Pxx[Gbounds[0]:Gbounds[-1]+1,:].mean(axis=0)
    #DpwrNormal = DavgPwr*Dmean
    #TpwrNormal = TavgPwr*Tmean
    #GpwrNormal = GavgPwr*Gmean
    DpwrNormal = pxxFlat[Dbounds[0]:Dbounds[-1]+1,:].mean(axis=0)
    TpwrNormal = pxxFlat[Tbounds[0]:Tbounds[-1]+1,:].mean(axis=0)
    GpwrNormal = pxxFlat[Gbounds[0]:Gbounds[-1]+1,:].mean(axis=0)
    ###Clustering states using k-means
    pwrData = np.array([DpwrNormal,TpwrNormal,GpwrNormal]).T
    
    f,ax=pl.subplots(1,1)
    ax.pcolormesh(t/2,frq,np.log10(pxxFlat),cmap='magma')
    #pl.plot(t/2,zScore(TpwrNormal))
    #ax.plot(traj.times*1e3- traj.times[0]*1e3,1+zScore(traj.places[:,0])+3,lw=1)
    #ax.set_xlim(0,3.5e5)

LOADING /home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/02-begin1/CSC02.lfp


/home/chenani/anaconda/lib/python2.7/site-packages/spectrum-0.6.1-py2.7-linux-x86_64.egg/spectrum/psd.py:451: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.__psd == None:


In [8]:
T,S = traj.getSpeed()
S = np.clip(S,S.mean(),S.mean()+3*S.std())
pl.plot(t/2,zScore(DpwrNormal))
pl.plot(t/2,zScore(TpwrNormal))
pl.plot(t/2,zScore(GpwrNormal))
pl.plot((T- lfp.timeAxis[0]/1e3)*1e3,MA(S,6),lw=1)

In [210]:
pl.plot(t/2,zScore(DpwrNormal- TpwrNormal))
pl.plot((T- lfp.timeAxis[0]/1e3)*1e3,3*MA(S,10),lw=1)
pl.axhline(0)

In [8]:
#data = sp.data_cosine(N=512, A=0.1, sampling=2000, freq=200)
data = lfp.signal_white[12000:12512]
# If you already have the DPSS windows
[tapers, eigen] = sp.dpss(512, 2, 4)
res = sp.pmtm(data, e=tapers, v=eigen,show=False)
# You do not need to compute the DPSS before end
pl.plot(p.frequencies(),res[:len(p.frequencies())])
#pl.plot(p.frequencies(),p.psd)
pl.yscale('log')

NameError: name 'p' is not defined

In [139]:
p = sp.Periodogram(data,sampling=2000)
p();
pl.plot(p.frequencies(),p.psd)